# Colab setup

In [ ]:
import sys
if "google.colab" in str(get_ipython()):
    ! {sys.executable} -m pip install pytorch-lifestream
    ! {sys.executable} -m pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
 

## Libs

In [ ]:
import os

import torch
import pickle
import numpy as np
import pandas as pd
import torchmetrics
from functools import partial
import pytorch_lightning as pl

from ptls.nn import Head
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames import PtlsDataModule
from ptls.frames.supervised import SequenceToTarget
from ptls.frames.supervised import SeqToTargetDataset
from ptls.preprocessing import PandasDataPreprocessor
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.datasets import inference_data_loader
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, metrics

## Data preproccessing

In [ ]:
path = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/transactions_train.csv.gz?download=true"
source_data = pd.read_csv(path, compression="gzip")
source_data

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [ ]:
def trx_transform(df):
    df["event_time"] = df["trans_date"].astype(float)
    df["weekday"] = df["trans_date"] % 7
    return df

source_data = trx_transform(source_data)

In [ ]:
preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="trans_date",
    event_time_transformation="none",
    cols_category=["small_group", "weekday"],
    category_transformation="frequency",
    cols_numerical=["amount_rur"],
    return_records=True,
)

In [ ]:
dataset = preprocessor.fit_transform(source_data)
dataset = sorted(dataset, key=lambda x: x["client_id"])

In [ ]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
path = "https://huggingface.co/datasets/dllllb/age-group-prediction/resolve/main/train_target.csv?download=true"
df_target = pd.read_csv(path)
df_target.set_index("client_id", inplace=True)
df_target.rename(columns={"bins": "target"}, inplace=True)
df_target

,target
client_id,
24662,2
1046,0
34089,2
34848,1
47076,3
...,...
14303,1
22301,2
25731,0


In [ ]:
for el in dataset:
    el["target"] = df_target["target"][el["client_id"]]

print(dataset[0].keys())

dict_keys(['client_id', 'trans_date', 'event_time', 'small_group', 'weekday', 'amount_rur', 'target'])


In [ ]:
train, test = train_test_split(dataset, test_size=0.2, random_state=42)
train, valid = train_test_split(train, test_size=0.1, random_state=42)

## FineTuning

### load SequenceEncoder obtained from `coles-emb.ipynb`

In [ ]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={"amount_rur": "identity"},
    embeddings={
        "trans_date": {"in": 800, "out": 16},
        "small_group": {"in": 250, "out": 16},
    },
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type="gru",
)

seq_encoder.load_state_dict(torch.load("/content/coles-emb.pt"))

<All keys matched successfully>

### model

In [ ]:
downstream_model = SequenceToTarget(
    seq_encoder=seq_encoder,
    head=Head(
        input_size=seq_encoder.embedding_size,
        use_batch_norm=True,
        objective="classification",
        num_classes=4,
    ),
    loss=torch.nn.NLLLoss(),
    metric_list=torchmetrics.Accuracy(task="multiclass", num_classes=4),
    pretrained_lr=0.0001,
    optimizer_partial=partial(torch.optim.Adam, lr=0.02),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.2),
)

### Data module

In [ ]:
def get_dataset(data):
    return SeqToTargetDataset(
        MemoryMapDataset(
            data=data,
        ),
        target_col_name="target",
    )

finetune_dm = PtlsDataModule(
    train_data=get_dataset(train),
    valid_data=get_dataset(valid),
    test_data=get_dataset(test),
    train_num_workers=4,
    train_batch_size=256,
    test_batch_size=128,
)

### Trainer FineTuning

In [ ]:
trainer_ft = pl.Trainer(
    max_epochs=4,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    enable_progress_bar=False,
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


### Training FineTuning

In [ ]:
trainer_ft.fit(downstream_model, finetune_dm)
print(trainer_ft.logged_metrics)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | seq_encoder   | RnnSeqEncoder | 240 K  | train
1 | head          | Head          | 1.5 K  | train
2 | loss          | NLLLoss       | 0      | train
3 | train_metrics | ModuleDict    | 0      | train
4 | valid_metrics | ModuleDict    | 0      | train
5 | test_metrics  | ModuleDict    | 0      | train
--------------------------------------------------------
242 K     Trainable params
0         Non-trainable params
242 K     Total params
0.968     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, 

{'loss': tensor(0.9243), 'seq_len': tensor(874.9375), 'y': tensor(1.4688), 'val_loss': tensor(0.9168), 'valid/MulticlassAccuracy': tensor(0.5850)}


### Testing

In [ ]:
trainer_ft.test(
    downstream_model,
    dataloaders=finetune_dm.test_dataloader()
)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test/MulticlassAccuracy  │    0.6111666560173035     │
└───────────────────────────┴───────────────────────────┘

[{'test/MulticlassAccuracy': 0.6111666560173035}]